In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
from PIL import Image
import os, glob
from tqdm import tqdm
import torchvision
from torchvision import datasets, transforms
from matplotlib.pyplot import imshow
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

from model import get_model

## Load Dataset

In [2]:
image_size = 224
batch_size = 16
num_workers = 4

In [3]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
# transform_train = transforms.Compose([transforms.Resize((image_size, image_size), interpolation=Image.ANTIALIAS), \
#                       transforms.RandomHorizontalFlip(),
#                       transforms.ToTensor(), 
#                       normalize])
transform_test = transforms.Compose([transforms.Resize((image_size, image_size), interpolation=Image.ANTIALIAS), \
                      transforms.ToTensor(), 
                      normalize])

# data_train = datasets.ImageFolder('/scratch/rw2268/Handbag_brand/data/train', transform=transform_train)
# train_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, 
#                                                          shuffle=True, 
#                                                          num_workers=num_workers)

# data_val = datasets.ImageFolder('/scratch/rw2268/Handbag_brand/data/val', transform=transform_test)
# val_loader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, 
#                                                      shuffle=False, 
#                                                      num_workers=num_workers)

# data_test = datasets.ImageFolder('/scratch/rw2268/Handbag_brand/data/test', transform=transform_test)
# test_loader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, 
#                                                        shuffle=False, 
#                                                        num_workers=num_workers)


data_test = datasets.ImageFolder("/Users/yuxiong/Desktop/Capstone/Fashion_Apparel_Detection/Capstone_Project/CNN-Classifier-Pytorch/data/test", transform=transform_test)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, 
                                                       shuffle=False, 
                                                       num_workers=num_workers)

In [4]:
brand=['Louis_Vuitton',
 'balenciaga',
 'burberry',
 'chloe',
 'coach',
 'givenchy',
 'gucci',
 'longchamp',
 'michaelkors',
 'prada',
 'saint laurent',
 'unknown',
 'valentino']

In [5]:
model_path= "/Users/yuxiong/Desktop/Capstone/Fashion_Apparel_Detection/Capstone_Project/yolov3/0012.pth"

In [6]:
def load_pretrained_model(model):
    try:
    
        model.load_state_dict(torch.load(model_path, map_location=lambda storage, loc: storage))

    except:
        return 0
    print('Loaded trained model: {}!'.format(model_path))
    #return int(os.path.basename(dir_).split('.')[0])

In [7]:
model = get_model(model_path, 13) 

The number of parameters /USERS/YUXIONG/DESKTOP/CAPSTONE/FASHION_APPAREL_DETECTION/CAPSTONE_PROJECT/YOLOV3/0012.PTH - ImageNet (1000 outputs): 11689512
The number of parameters /USERS/YUXIONG/DESKTOP/CAPSTONE/FASHION_APPAREL_DETECTION/CAPSTONE_PROJECT/YOLOV3/0012.PTH - Custom (13 outputs): 11696181


In [8]:
optimizer = torch.optim.Adam(model.parameters(), 0.001, [0.5, 0.999])

In [9]:
load_pretrained_model(model)

Loaded trained model: /Users/yuxiong/Desktop/Capstone/Fashion_Apparel_Detection/Capstone_Project/yolov3/0012.pth!


In [10]:
def solver(data_loader, model):
    model.eval()
   
    count_test = 0
    test_out = []
    
    for batch_idx, (data,target) in enumerate(data_loader):
        #data = Variable(data, volatile=True)
        output = model(data)
        _, arg_max_out = torch.max(output.data.cpu(), 1)
        
        
        for oo in arg_max_out:
            test_out.append('%s,%d\n'%(str(count_test).zfill(4), oo))
            count_test+=1

    #print(brand[int(test_out[0].split(",")[1])])

    return brand[int(test_out[0].split(",")[1])]

def test(test_loader, model):
    #assert start_epoch>0, "you must first TRAIN"
    #solver(config.model, val_loader, model,  mode='val')
    return solver(test_loader, model)



In [11]:
outcome = test(test_loader, model)

In [12]:
outcome

'gucci'

In [ ]:
num_classes=len(train_loader.dataset.classes)
num_classes
train_loader.dataset.classes

In [ ]:
sp_img = data_train.__getitem__(345)[0].numpy()
# img = Image.fromarray(sp_img, 'RGB')
sp_img = sp_img.transpose([1,2,0])
imshow(sp_img)

## Load a pre-trained CNN and Train model

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
num_epochs = 10
num_epochs_decay = 60
learning_rate = 0.001
stop_training = 3
def update_lr(lr, optimizer):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
def remove_layer(model, num_classes):
    modules = model.modules()
    for m in modules:
        if isinstance(m, nn.Linear) and m.weight.data.size()[0]==1000:
            w1 = m.weight.data[:num_classes]
            b1 = m.bias.data[:num_classes]
    try:
        if type(model.classifier)==nn.Sequential:
            mod = list(model.classifier) #Alexnet, VGG
        else:
            mod = [model.classifier] #DenseNet

    except: 
        mod = [model.fc] #ResNet

    weight = mod[-1].weight.size(1)
    mod.pop()
    mod.append(torch.nn.Linear(weight,num_classes))
    new_classifier = torch.nn.Sequential(*mod)
    model.classifier = new_classifier
    modules = model.modules()
    flag = False
    for m in modules:
        if isinstance(m, nn.Linear) and m.weight.data.size()[0]==num_classes:
            m.weight.data = w1
            m.bias.data = b1  
            flag = True
    assert flag

In [ ]:
model = torchvision.models.resnet18(pretrained=True)
remove_layer(model, num_classes)

In [ ]:
def save_model(model, name, epoch):
    dir_ = os.path.join('/scratch/rw2268/Handbag_brand/snapshot', name, '%s.pth'%(str(epoch).zfill(4)))
    create_folder(os.path.dirname(dir_))
    torch.save(model.state_dict(), dir_)
    print('!!Saving model: {}!'.format(dir_))
def create_folder(folder):
    if not os.path.isdir(folder):
        os.makedirs(folder)

In [ ]:
def solver(name, data_loader, model, epoch, optimizer=None, mode='train'):
    model = model.to(device)
    if optimizer is None: model.eval()
    else: model.train()
    loss_cum = []
    Acc = 0
    count_test = 0
    test_out = []
    Loss = nn.CrossEntropyLoss()
    for batch_idx, (data,target) in tqdm(enumerate(data_loader), 
                total=len(data_loader), desc="!{} -> [{}] Epoch: {}".format(name.upper(), mode.upper(),epoch)):
        volatile = True if optimizer is None else False
        data = Variable(data.to(device), volatile=volatile)
        target = Variable(target.to(device), volatile=volatile)
       
        output = model(data)
        loss = Loss(output,target)   
        
        if optimizer is not None:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
       

        loss_cum.append(loss.data.cpu()[0])
        _, arg_max_out = torch.max(output.data.cpu(), 1)
      
        if mode=='test':
            for oo in arg_max_out:
                test_out.append('%s,%d\n'%(str(count_test).zfill(4), oo))
                count_test+=1

        Acc += arg_max_out.long().eq(target.data.cpu().long()).sum()
#         if batch_idx % 20 == 0:
#             print('Epoch:{}, batch:{}, Acc:{}'.format(epoch+1,batch_idx+1,Acc*100/)
    ACC = float(Acc*100)/len(data_loader.dataset)
    LOSS = np.array(loss_cum).mean()
    if mode=='test':
        f=open(os.path.join('snapshot', name, 'test.txt'),'w')
        for line in test_out: f.writelines(line)
        f.close()
    else:
        print("LOSS %s: %0.3f || ACC %s: %0.2f"%(mode.upper(), LOSS, mode.upper(), ACC))
    
    return ACC

#==========================================================================#
def train(name,train_loader, val_loader, model, learning_rate=learning_rate):
    val_before = 0
    optimizer = torch.optim.Adam(model.parameters(), learning_rate, [0.5, 0.999])
    model.train()
    for epoch in range(num_epochs):
        solver(name, train_loader, model, epoch, optimizer=optimizer, mode='train')
        val_acc = solver(name, val_loader, model, epoch, mode='val')
        if val_acc>val_before:
            save_model(model, name, epoch+1)
            val_before=val_acc
            flag_stop=0
        else:
            flag_stop+=1

        if flag_stop==stop_training: 
            return

    # Decay learning rate
        if (epoch+1) > (num_epochs - num_epochs_decay):
            learning_rate -= (learning_rate / float(num_epochs_decay))
            update_lr(learning_rate, optimizer)
            print ('Decay learning rate to: {}.'.format(learning_rate))

In [ ]:
train('ResNet18',train_loader, val_loader, model)

In [ ]:
device